In [1]:
import urllib
import json
import urllib.request as urllib2
import pandas as pd
import csv
import sys
import math
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import colorama
from colorama import Fore

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

# CESMD USER QUERY NOTEBOOK (Earthquake, Station, Record)
### - **Default format is CSV for event and station. Record is using json format. (editable**)

### **Parameters and description**

| Parameters ||Description |  ||
| :- | -: || -: | -: | 
|Circle||Return events within a given distance (in kilometers) of a location|
|Coordinates||Return stations within a range of latitudes or longitudes|
|Country/State||Return events in a given country or U.S. state|
|Date/Time||Search by date and time (in UTC) in yyyy-mm-dd hh:mm:ss format|
|Earthquake Name||May contain the nearby city to the epicenter. A search for a nearby city returns events whose earthquake name contains the name of the nearby city. |
|Epicentral Dist. (km)||Specify an epicentral distance range (min to max inclusive) to narrow down search criteria|
|Event IDs||Search event(s) by ID(s), e.g. CI37904927,CI38043999. Most other options are unavailable when an event ID is specified. |
|Fault Type||Return events with specific fault type. Multiple fault types may be selected for the search. Leaving them unselected will search for all possible types. |
|Group By||Specify the grouping of the metadata. Default is grouping by stations. Grouping by stations will group records under the same station into a single data structure. |
|Include Inactive Stations||Check to include inactive stations in search criteria, uncheck otherwise. |
|Login Email||Available if dataset is selected as the return type. Registered e-mail must be provided in order to download data files. |
|Metadata Format||Specify output format. Default is JSON. Available if metadata is selected as the return type. |
|Magnitude||Self-explanatory. Returns events within a range of magnitudes. Default returns all magnitude ranges|
|Network||Two letter Network ID of station (multiple network IDs can be listed, e.g. CE,AA) |
|Order Records By||Specifies the sort order of the record output. Default is by epicentral distance (descending). |
|PGA (g)||Specify a PGA range (min to max inclusive) to narrow down search criteria|
|Processed, Raw, Plot||Available if dataset is selected as the return type. Checking processed and raw will package both processed and raw data in the returned (.ZIP) file. |
|Return Type||Return type of the output. Dataset will return a (.ZIP) file packaged with the requested data. Metadata will return either (.JSON) or (.XML), depending on user selection. |
|Station Code(s)||Search station(s) by code(s), e.g. CE24319,AAAK16. Most other options are unavailable when a station code is specified. |
|Station Name||Name of station, accepts wildcards|
|Station Type||Default is Any, alternative options are Array, Ground, Building, Bridge, Dam, Tunnel, Wharf, Other|


In [2]:
country_list = ["US","British Virgin Islands","Canada","Chile",
        "Costa Rica","Cuba","Ecuador", "Guatemala","Haiti","Indonesia","Italy",
        "Japan","Kermadec Islands","Mexico","Nepal","New Zealand","Papua New Guinea",
        "Puerto Rico","Solomon Islands","Spain", "Taiwan", "Tonga","Turkey","Vanuatu"]
state_list = ["AK","AR","CA","DE","HI","ID","IL","KS","MT","NV","OK","OR","SC","VA","WA"]

In [3]:
"""LOCATION QUERY FUNCTION"""
# By Country
def earthquake_country(url):
    print('Country List:\n',country_list)
    country= input('What is the country?')  
    if country != '':
        url = url+'&country='+country
        print('State List:\n',state_list)
        state= input('what is the state?') 
        if state != '':
            url = url+'&state='+state
        else: 
            print('No input state.')
    else: 
        print('No input country.')    
    print(url)
    return url




# By Circle
def earthquake_country_circle(url):
    latitue = input('Latitude at circle center')  
    longitude = input('Longitude at circle center')
    radius = input ('Radius(km) from circle center')
    if  latitue and longitude and radius: 
        url = url+f'&lat={latitue}&lon={longitude}&rad={radius}'
        print(url)
    else:
        print(Fore.RED + 'You may missing at least one parameter, the url will not contain the circle query')
        print(url)
    return url




# By Coordinates
def earthquake_Coordinates(url):
    minlat=input('Min Latitude') 
    maxlat=input('Max Latitude') 
    minlon=input('Min Longitude')
    maxlon=input('Max Longitude')
    
    if  minlat and maxlat and minlon and maxlon: 
        url = url +f'&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
        print(url)
    else:
        print(Fore.RED + 'You may missing at least one parameter, the url will not contain the coordinates query')
        print(url)
    return url

In [6]:
# Start with create the query url

def earthquake_parameter(url):
    # Start enter the input and check the format to make url
    print("For the following question, please follow the instructions or skip by pushing the 'return' button. Thanks!\n")
    #
    minmag= input('Min magnitude   (a number between 0~10):')
    maxmag= input('Max magnitude   (a number between 0~10, greater than min magnitude):')
    startdate= input('Enter a start date (ex.2021-08-25, 2021-08-14T11:57:43.715Z):')    
    enddate=input('Enter a end date (ex.2021-08-25, 2021-08-14T11:57:43.715Z, later than start date):')
    faulttype = input('Enter a fault type (ex.NM,RS,SS. Default is any):')

    
    # Eroor Handeling
    if minmag != '' and float(minmag):
        url = url+'&minmag='+minmag
    else: 
        print(Fore.RED +'No min magnitude or input is not a number.')
        

    if maxmag != '' and float(maxmag):
        if minmag == '' or float(maxmag) >= float(minmag):
            url = url+'&maxmag='+maxmag
        else:
            print(Fore.RED +'Max magnitude is not a number.')
    else: 
        print(Fore.RED +'No max magnitude.')

    if startdate != '':
        url = url+'&startdate='+startdate
    else: 
        print(Fore.RED +'No input start date.')

    if enddate != '':
        url = url+'&enddate='+enddate
    else: 
        print(Fore.RED +'No input end date.')    
    
    if faulttype != '':
        url = url+'&faulttype='+faulttype
    else:
        print(Fore.RED +'fault type is any')
        

    #orderby=time
    #format=csv
    print(url)
    return url






# ask users input and create the complete URL by adding pieces 
def query_earthquake(url):
    event_id = input('Do you have an id number? (Yes/No)')
    # event id is the primary 
    if  event_id.lower() == 'yes':
        id_ = input('Event ID:')
        url = url +f'&eventid={id_}'
    # parameters and location to the url 
    else:
        query = input('\nDo you want to query by parameters, location, or both? (parameters/location/both)')
        if query.lower() == 'parameters':
            url = earthquake_parameter(url)
        elif query.lower() == 'location':
            e,s='',''
            loc_type= input('Do you want to query by country, circle or coordination?')
            if loc_type.lower().lower() == 'country':
                url = earthquake_country(url)
            elif loc_type.lower() == 'circle':
                url =  earthquake_country_circle(url)
            elif loc_type.lower() == 'coordination':
                url = earthquake_Coordinates(url)
            else:
                print(Fore.RED+'No input for location')
        elif query.lower() == 'both':
            url = earthquake_parameter(url)
            loc_type= input('Do you want to query by country, circle or coordination?')
            if loc_type.lower() == 'country':
                url = earthquake_country(url)
            elif loc_type.lower() == 'circle':
                url =  earthquake_country_circle(url)
            elif loc_type.lower() == 'coordination':
                url = earthquake_Coordinates(url)
            else:
                print('No input for location')
        else:
            print(Fore.RED +('Unknow input, you may need try again if the url is not working'))
    if url == 'https://www.strongmotioncenter.org/wserv/events/query?&orderby=time&format=csv&nodata=404':
        print(Fore.RED +'You must enter at least one parameter! Try Again.')
    
    return url 

In [5]:
def open_url(url):
    try:
        if 'eventid=' in event_url:
            data = pd.read_csv(url, sep="|", engine="python", skiprows=5)
        else:
            data = pd.read_csv(url, sep="|", engine="python", skiprows=4)
    except:
        data = []
        print(Fore.RED +'URL show Eroor, please check inputs')
    return data

# Query Event

In [ ]:
# Sample Query URL 
url = 'https://www.strongmotioncenter.org/wserv/events/query?&orderby=time&format=csv&nodata=404' # initial url with csv format
event_url = query_earthquake(url)

In [ ]:
data = open_url(event_url)
print(data.describe())
data.head()

In [ ]:
#data.to_csv('event.csv') 

### Plot the data in a map

In [ ]:
def plot_map(data,lat_0=38.1,lon_0=-118,zoom_start = 3,lon='Longitude',lat='Latitude'):
    """
    data: the dataframe
    lat_0,lon_0, the center of the map
    zoom_start
    lon & lat: the columns name for longtitude and latitude
    """
    # Create a map
    m_1 = folium.Map(location=[lat_0,lon_0], tiles='openstreetmap', zoom_start=3)
    # Add points to the map
    mc = MarkerCluster()
    for idx, row in data.iterrows():
        if not math.isnan(row[lon]) and not math.isnan(row[lat]):
            mc.add_child(Marker([row[lat], row[lon]]))
    m_1.add_child(mc)

    # Display the map
    return m_1

In [ ]:
m_1 = plot_map(data,lat_0=38.1,lon_0=-118,zoom_start = 3,lon='Longitude',lat='Latitude')
m_1